In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imsave
from PIL import Image
import numpy as np
import os

In [69]:
# Train Dataset
X = []
for filename in os.listdir('./Train/'):
    image = torchvision.io.read_image('./Train/'+filename)
    # image = np.array(X, dtype=float)
    X.append(image)
    # print(image.shape)
# print(X)
X = np.array(X, dtype=float)
# print(X)4

# Test Dataset
Y = []
for filename in os.listdir('./Test/'):
    image = torchvision.io.read_image('./Test/'+filename)
    Y.append(image)
Y = np.array(Y, dtype=float)


In [70]:
# Preparing test and train data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [71]:
# Model defining
class ColorModel(nn.Module):
    def __init__(self):
        super(ColorModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, 3,padding=1),  # Assuming grayscale input images
            nn.ReLU(),
            nn.Conv2d(64, 64, 3,padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3,padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3,padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3,padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3,padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 2, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Upsample(scale_factor=2)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x * 128.0


In [72]:
batch_size = 10
learning_rate = 0.001
epochs = 1
folder_path_train = './Train/'
folder_path_test = './Test/'

data_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = X
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [73]:
# Initialize model
model = ColorModel()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

In [96]:
# Training Loop
for epoch in range(epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()
        lab_batch = rgb2lab(np.array(batch))
        X_batch = lab_batch[:, :, :, 0]
        Y_batch = lab_batch[:, :, :, 1:] / 128
        X_batch = torch.Tensor(X_batch).unsqueeze(1)  # Add channel dimension
        Y_batch = torch.Tensor(Y_batch)
        
        # Forward pass
        output = model(X_batch)
        
        # Compute loss
        loss = criterion(output, Y_batch)
        
        # Backward pass
        loss.backward()
        optimizer.step()

ValueError: the input array must have size 3 along `channel_axis`, got (8, 3, 256, 256)

In [98]:
test_dataset = ColorModel(folder_path_test, transform=data_transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(test_dataloader):
        lab_batch = rgb2lab(np.array(batch))
        X_batch = lab_batch[:, :, :, 0]
        X_batch = torch.Tensor(X_batch).unsqueeze(1)  # Add channel dimension
        output = model(X_batch)
        output = output * 128

        # Output colorizations
        for j in range(output.size(0)):
            cur = np.zeros((256, 256, 3))
            cur[:, :, 0] = X_batch[j, 0].numpy()
            cur[:, :, 1:] = output[j].numpy()
            imsave(f"result/img_{i * batch_size + j}.png", lab2rgb(cur))

TypeError: ColorModel.__init__() got an unexpected keyword argument 'transform'